In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [930 kB]
Hit:13 http://ppa.laun

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-02-26 18:59:29--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.22MB/s    in 0.8s    

2022-02-26 18:59:31 (1.22 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("formula1ETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://formula1-team8-bucket.s3.amazonaws.com/circuits.csv"
spark.sparkContext.addFile(url)
circuits_df = spark.read.csv(SparkFiles.get("circuits.csv"), sep=",", header=True)

In [5]:
# Show DataFrame
circuits_df.show()

+---------+--------------+--------------------+------------+---------+--------+---------+---+--------------------+
|circuitId|    circuitRef|                name|    location|  country|     lat|      lng|alt|                 url|
+---------+--------------+--------------------+------------+---------+--------+---------+---+--------------------+
|        1|   albert_park|Albert Park Grand...|   Melbourne|Australia|-37.8497|  144.968| 10|http://en.wikiped...|
|        2|        sepang|Sepang Internatio...|Kuala Lumpur| Malaysia| 2.76083|  101.738| 18|http://en.wikiped...|
|        3|       bahrain|Bahrain Internati...|      Sakhir|  Bahrain| 26.0325|  50.5106|  7|http://en.wikiped...|
|        4|     catalunya|Circuit de Barcel...|    Montmeló|    Spain|   41.57|  2.26111|109|http://en.wikiped...|
|        5|      istanbul|       Istanbul Park|    Istanbul|   Turkey| 40.9517|   29.405|130|http://en.wikiped...|
|        6|        monaco|   Circuit de Monaco| Monte-Carlo|   Monaco| 43.7347| 

In [6]:
url ="https://formula1-team8-bucket.s3.amazonaws.com/constructor_results.csv"
spark.sparkContext.addFile(url)
constructor_results_df = spark.read.csv(SparkFiles.get("constructor_results.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
constructor_results_df.show()

+--------------------+------+-------------+------+------+
|constructorResultsId|raceId|constructorId|points|status|
+--------------------+------+-------------+------+------+
|                   1|    18|            1|  14.0|    \N|
|                   2|    18|            2|   8.0|    \N|
|                   3|    18|            3|   9.0|    \N|
|                   4|    18|            4|   5.0|    \N|
|                   5|    18|            5|   2.0|    \N|
|                   6|    18|            6|   1.0|    \N|
|                   7|    18|            7|   0.0|    \N|
|                   8|    18|            8|   0.0|    \N|
|                   9|    18|            9|   0.0|    \N|
|                  10|    18|           10|   0.0|    \N|
|                  11|    18|           11|   0.0|    \N|
|                  12|    19|            6|  10.0|    \N|
|                  13|    19|            2|  11.0|    \N|
|                  14|    19|            1|  10.0|    \N|
|             

In [7]:
url ="https://formula1-team8-bucket.s3.amazonaws.com/constructor_standings.csv"
spark.sparkContext.addFile(url)
constructor_standings_df = spark.read.csv(SparkFiles.get("constructor_standings.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
constructor_standings_df.show()

+----------------------+------+-------------+------+--------+------------+----+
|constructorStandingsId|raceId|constructorId|points|position|positionText|wins|
+----------------------+------+-------------+------+--------+------------+----+
|                     1|    18|            1|  14.0|       1|           1|   1|
|                     2|    18|            2|   8.0|       3|           3|   0|
|                     3|    18|            3|   9.0|       2|           2|   0|
|                     4|    18|            4|   5.0|       4|           4|   0|
|                     5|    18|            5|   2.0|       5|           5|   0|
|                     6|    18|            6|   1.0|       6|           6|   0|
|                     7|    19|            1|  24.0|       1|           1|   1|
|                     8|    19|            2|  19.0|       2|           2|   0|
|                     9|    19|            3|   9.0|       4|           4|   0|
|                    10|    19|         

In [8]:
url ="https://formula1-team8-bucket.s3.amazonaws.com/constructors.csv"
spark.sparkContext.addFile(url)
constructors_df = spark.read.csv(SparkFiles.get("constructors.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
constructors_df.show()

+-------------+--------------+-----------+-----------+--------------------+
|constructorId|constructorRef|       name|nationality|                 url|
+-------------+--------------+-----------+-----------+--------------------+
|            1|       mclaren|    McLaren|    British|http://en.wikiped...|
|            2|    bmw_sauber| BMW Sauber|     German|http://en.wikiped...|
|            3|      williams|   Williams|    British|http://en.wikiped...|
|            4|       renault|    Renault|     French|http://en.wikiped...|
|            5|    toro_rosso| Toro Rosso|    Italian|http://en.wikiped...|
|            6|       ferrari|    Ferrari|    Italian|http://en.wikiped...|
|            7|        toyota|     Toyota|   Japanese|http://en.wikiped...|
|            8|   super_aguri|Super Aguri|   Japanese|http://en.wikiped...|
|            9|      red_bull|   Red Bull|   Austrian|http://en.wikiped...|
|           10|   force_india|Force India|     Indian|http://en.wikiped...|
|           

In [9]:
url ="https://formula1-team8-bucket.s3.amazonaws.com/driver_standings.csv"
spark.sparkContext.addFile(url)
driver_standings_df = spark.read.csv(SparkFiles.get("driver_standings.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
driver_standings_df.show()

+-----------------+------+--------+------+--------+------------+----+
|driverStandingsId|raceId|driverId|points|position|positionText|wins|
+-----------------+------+--------+------+--------+------------+----+
|                1|    18|       1|  10.0|       1|           1|   1|
|                2|    18|       2|   8.0|       2|           2|   0|
|                3|    18|       3|   6.0|       3|           3|   0|
|                4|    18|       4|   5.0|       4|           4|   0|
|                5|    18|       5|   4.0|       5|           5|   0|
|                6|    18|       6|   3.0|       6|           6|   0|
|                7|    18|       7|   2.0|       7|           7|   0|
|                8|    18|       8|   1.0|       8|           8|   0|
|                9|    19|       1|  14.0|       1|           1|   1|
|               10|    19|       2|  11.0|       3|           3|   0|
|               11|    19|       3|   6.0|       6|           6|   0|
|               12| 

In [10]:
url ="https://formula1-team8-bucket.s3.amazonaws.com/drivers.csv"
spark.sparkContext.addFile(url)
drivers_df = spark.read.csv(SparkFiles.get("drivers.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
drivers_df.show()

+--------+----------+------+----+---------+----------+----------+-----------+--------------------+
|driverId| driverRef|number|code| forename|   surname|       dob|nationality|                 url|
+--------+----------+------+----+---------+----------+----------+-----------+--------------------+
|       1|  hamilton|    44| HAM|    Lewis|  Hamilton|1985-01-07|    British|http://en.wikiped...|
|       2|  heidfeld|    \N| HEI|     Nick|  Heidfeld|1977-05-10|     German|http://en.wikiped...|
|       3|   rosberg|     6| ROS|     Nico|   Rosberg|1985-06-27|     German|http://en.wikiped...|
|       4|    alonso|    14| ALO| Fernando|    Alonso|1981-07-29|    Spanish|http://en.wikiped...|
|       5|kovalainen|    \N| KOV|   Heikki|Kovalainen|1981-10-19|    Finnish|http://en.wikiped...|
|       6|  nakajima|    \N| NAK|   Kazuki|  Nakajima|1985-01-11|   Japanese|http://en.wikiped...|
|       7|  bourdais|    \N| BOU|Sébastien|  Bourdais|1979-02-28|     French|http://en.wikiped...|
|       8|

In [11]:
url ="https://formula1-team8-bucket.s3.amazonaws.com/lap_times.csv"
spark.sparkContext.addFile(url)
lap_times_df = spark.read.csv(SparkFiles.get("lap_times.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
lap_times_df.show()

+------+--------+---+--------+--------+------------+
|raceId|driverId|lap|position|    time|milliseconds|
+------+--------+---+--------+--------+------------+
|   841|      20|  1|       1|1:38.109|       98109|
|   841|      20|  2|       1|1:33.006|       93006|
|   841|      20|  3|       1|1:32.713|       92713|
|   841|      20|  4|       1|1:32.803|       92803|
|   841|      20|  5|       1|1:32.342|       92342|
|   841|      20|  6|       1|1:32.605|       92605|
|   841|      20|  7|       1|1:32.502|       92502|
|   841|      20|  8|       1|1:32.537|       92537|
|   841|      20|  9|       1|1:33.240|       93240|
|   841|      20| 10|       1|1:32.572|       92572|
|   841|      20| 11|       1|1:32.669|       92669|
|   841|      20| 12|       1|1:32.902|       92902|
|   841|      20| 13|       1|1:33.698|       93698|
|   841|      20| 14|       3|1:52.075|      112075|
|   841|      20| 15|       4|1:38.385|       98385|
|   841|      20| 16|       2|1:31.548|       

In [12]:
url ="https://formula1-team8-bucket.s3.amazonaws.com/pit_stops.csv"
spark.sparkContext.addFile(url)
pit_stops_df = spark.read.csv(SparkFiles.get("pit_stops.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
pit_stops_df.show()

+------+--------+----+---+--------+--------+------------+
|raceId|driverId|stop|lap|    time|duration|milliseconds|
+------+--------+----+---+--------+--------+------------+
|   841|     153|   1|  1|17:05:23|  26.898|       26898|
|   841|      30|   1|  1|17:05:52|  25.021|       25021|
|   841|      17|   1| 11|17:20:48|  23.426|       23426|
|   841|       4|   1| 12|17:22:34|  23.251|       23251|
|   841|      13|   1| 13|17:24:10|  23.842|       23842|
|   841|      22|   1| 13|17:24:29|  23.643|       23643|
|   841|      20|   1| 14|17:25:17|  22.603|       22603|
|   841|     814|   1| 14|17:26:03|  24.863|       24863|
|   841|     816|   1| 14|17:26:50|  25.259|       25259|
|   841|      67|   1| 15|17:27:34|  25.342|       25342|
|   841|       2|   1| 15|17:27:41|  22.994|       22994|
|   841|       1|   1| 16|17:28:24|  23.227|       23227|
|   841|     808|   1| 16|17:28:39|  24.535|       24535|
|   841|       3|   1| 16|17:29:00|  23.716|       23716|
|   841|     1

In [13]:
url ="https://formula1-team8-bucket.s3.amazonaws.com/qualifying.csv"
spark.sparkContext.addFile(url)
qualifying_df = spark.read.csv(SparkFiles.get("qualifying.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
qualifying_df.show()

+---------+------+--------+-------------+------+--------+--------+--------+--------+
|qualifyId|raceId|driverId|constructorId|number|position|      q1|      q2|      q3|
+---------+------+--------+-------------+------+--------+--------+--------+--------+
|        1|    18|       1|            1|    22|       1|1:26.572|1:25.187|1:26.714|
|        2|    18|       9|            2|     4|       2|1:26.103|1:25.315|1:26.869|
|        3|    18|       5|            1|    23|       3|1:25.664|1:25.452|1:27.079|
|        4|    18|      13|            6|     2|       4|1:25.994|1:25.691|1:27.178|
|        5|    18|       2|            2|     3|       5|1:25.960|1:25.518|1:27.236|
|        6|    18|      15|            7|    11|       6|1:26.427|1:26.101|1:28.527|
|        7|    18|       3|            3|     7|       7|1:26.295|1:26.059|1:28.687|
|        8|    18|      14|            9|     9|       8|1:26.381|1:26.063|1:29.041|
|        9|    18|      10|            7|    12|       9|1:26.919

In [14]:
url ="https://formula1-team8-bucket.s3.amazonaws.com/races.csv"
spark.sparkContext.addFile(url)
races_df = spark.read.csv(SparkFiles.get("races.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
races_df.show()

+------+----+-----+---------+--------------------+----------+--------+--------------------+
|raceId|year|round|circuitId|                name|      date|    time|                 url|
+------+----+-----+---------+--------------------+----------+--------+--------------------+
|     1|2009|    1|        1|Australian Grand ...|2009-03-29|06:00:00|http://en.wikiped...|
|     2|2009|    2|        2|Malaysian Grand Prix|2009-04-05|09:00:00|http://en.wikiped...|
|     3|2009|    3|       17|  Chinese Grand Prix|2009-04-19|07:00:00|http://en.wikiped...|
|     4|2009|    4|        3|  Bahrain Grand Prix|2009-04-26|12:00:00|http://en.wikiped...|
|     5|2009|    5|        4|  Spanish Grand Prix|2009-05-10|12:00:00|http://en.wikiped...|
|     6|2009|    6|        6|   Monaco Grand Prix|2009-05-24|12:00:00|http://en.wikiped...|
|     7|2009|    7|        5|  Turkish Grand Prix|2009-06-07|12:00:00|http://en.wikiped...|
|     8|2009|    8|        9|  British Grand Prix|2009-06-21|12:00:00|http://en.

In [15]:
url ="https://formula1-team8-bucket.s3.amazonaws.com/results.csv"
spark.sparkContext.addFile(url)
results_df = spark.read.csv(SparkFiles.get("results.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
results_df.show()

+--------+------+--------+-------------+------+----+--------+------------+-------------+------+----+-----------+------------+----------+----+--------------+---------------+--------+
|resultId|raceId|driverId|constructorId|number|grid|position|positionText|positionOrder|points|laps|       time|milliseconds|fastestLap|rank|fastestLapTime|fastestLapSpeed|statusId|
+--------+------+--------+-------------+------+----+--------+------------+-------------+------+----+-----------+------------+----------+----+--------------+---------------+--------+
|       1|    18|       1|            1|    22|   1|       1|           1|            1|  10.0|  58|1:34:50.616|     5690616|        39|   2|      1:27.452|        218.300|       1|
|       2|    18|       2|            2|     3|   5|       2|           2|            2|   8.0|  58|     +5.478|     5696094|        41|   3|      1:27.739|        217.586|       1|
|       3|    18|       3|            3|     7|   7|       3|           3|            3|  

In [16]:
url ="https://formula1-team8-bucket.s3.amazonaws.com/seasons.csv"
spark.sparkContext.addFile(url)
seasons_df = spark.read.csv(SparkFiles.get("seasons.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
seasons_df.show()

+----+--------------------+
|year|                 url|
+----+--------------------+
|2009|https://en.wikipe...|
|2008|https://en.wikipe...|
|2007|https://en.wikipe...|
|2006|https://en.wikipe...|
|2005|https://en.wikipe...|
|2004|https://en.wikipe...|
|2003|https://en.wikipe...|
|2002|https://en.wikipe...|
|2001|https://en.wikipe...|
|2000|https://en.wikipe...|
|1999|https://en.wikipe...|
|1998|https://en.wikipe...|
|1997|https://en.wikipe...|
|1996|https://en.wikipe...|
|1995|https://en.wikipe...|
|1994|https://en.wikipe...|
|1993|https://en.wikipe...|
|1992|https://en.wikipe...|
|1991|https://en.wikipe...|
|1990|https://en.wikipe...|
+----+--------------------+
only showing top 20 rows



In [17]:
url ="https://formula1-team8-bucket.s3.amazonaws.com/status.csv"
spark.sparkContext.addFile(url)
status_df = spark.read.csv(SparkFiles.get("status.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
status_df.show()

+--------+------------+
|statusId|      status|
+--------+------------+
|       1|    Finished|
|       2|Disqualified|
|       3|    Accident|
|       4|   Collision|
|       5|      Engine|
|       6|     Gearbox|
|       7|Transmission|
|       8|      Clutch|
|       9|  Hydraulics|
|      10|  Electrical|
|      11|      +1 Lap|
|      12|     +2 Laps|
|      13|     +3 Laps|
|      14|     +4 Laps|
|      15|     +5 Laps|
|      16|     +6 Laps|
|      17|     +7 Laps|
|      18|     +8 Laps|
|      19|     +9 Laps|
|      20|    Spun off|
+--------+------------+
only showing top 20 rows



In [18]:
url ="https://formula1-team8-bucket.s3.amazonaws.com/points_collision.csv"
spark.sparkContext.addFile(url)
points_collision_df = spark.read.csv(SparkFiles.get("points_collision.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
points_collision_df.show()

+--------+-------------------+---------------+
|resultId|total_points_driver|average_col_acc|
+--------+-------------------+---------------+
|   23782|              636.0|           0.04|
|   23783|             1555.5|           0.01|
|   23784|              308.0|           0.01|
|   23785|              458.0|           0.04|
|   23786|              131.0|           0.05|
|   23787|             1136.5|           0.09|
|   23788|              116.0|           0.11|
|   23789|             1022.0|           0.04|
|   23790|               12.0|            0.0|
|   23791|              418.5|           0.04|
|   23792|              429.0|           0.03|
|   23793|              185.0|           0.07|
|   23794|              560.0|            0.1|
|   23795|              136.0|           0.08|
|   23796|                4.0|            0.1|
|   23797|               47.0|           0.11|
|   23798|               77.0|           0.05|
|   23799|              309.0|           0.06|
|   23800|   

In [19]:
url ="https://formula1-team8-bucket.s3.amazonaws.com/weather.csv"
spark.sparkContext.addFile(url)
weather_df = spark.read.csv(SparkFiles.get("weather.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
weather_df.show()

+------+--------------------+------------+
|raceId|             weather|weather_type|
+------+--------------------+------------+
|   989|Parzialmente nuvo...|           2|
|   990|              Sereno|           1|
|   991|          Soleggiato|           1|
|   992|            Nuvoloso|           2|
|   993|Parzialmente nuvo...|           2|
|   994|             Coperto|           2|
|   995|              Sereno|           1|
|   996|Parzialmente nuvo...|           2|
|   997|              Sereno|           1|
|   998|          Soleggiato|           1|
|   999|Parzialmente nuvo...|           3|
|  1000|              Sereno|           1|
|  1001|Parzialmente nuvo...|           2|
|  1002|Parzialmente nuvo...|           2|
|  1003|              Sereno|           1|
|  1004|Parzialmente nuvo...|           2|
|  1005|          Soleggiato|           1|
|  1006|          Soleggiato|           1|
|  1007|Parzialmente nuvo...|           2|
|  1008|Parzialmente nuvo...|           2|
+------+---

In [24]:
# Store environmental variable
from getpass import getpass
# password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://formula1-team8.cwip4lqac6n6.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": "postgresteam8",
          "driver":"org.postgresql.Driver"}

In [ ]:
circuits_df = circuits_df.withColumn("circuitId",circuits_df.circuitId.cast("int"))
circuits_df = circuits_df.withColumn("lat",circuits_df.lat.cast("float"))
circuits_df = circuits_df.withColumn("lng",circuits_df.lng.cast("float"))
circuits_df = circuits_df.withColumn("alt",circuits_df.alt.cast("int"))
drivers_df = drivers_df.withColumn("number",drivers_df.number.cast("int"))
drivers_df = drivers_df.withColumn("dob",drivers_df.dob.cast("date"))
races_df = races_df.withColumn("date",races_df.date.cast("date"))
races_df = races_df.withColumn("time",races_df.time.cast("Timestamp"))
pit_stops_df = pit_stops_df.withColumn("time",pit_stops_df.time.cast("Timestamp"))
results_df = results_df.withColumn("number",results_df.number.cast("int"))
results_df = results_df.withColumn("position",results_df.position.cast("int"))
results_df = results_df.withColumn("milliseconds",results_df.milliseconds.cast("int"))
results_df = results_df.withColumn("fastestLap",results_df.fastestLap.cast("int"))
results_df = results_df.withColumn("rank",results_df.rank.cast("int"))

In [22]:
weather_df.dtypes

[('raceId', 'int'), ('weather', 'string'), ('weather_type', 'int')]

In [ ]:
# Write DataFrame to active_user table in RDS
circuits_df.write.jdbc(url=jdbc_url, table='circuits', mode=mode, properties=config)

In [ ]:
drivers_df.write.jdbc(url=jdbc_url, table='drivers', mode=mode, properties=config)

In [ ]:
constructors_df.write.jdbc(url=jdbc_url, table='constructors', mode=mode, properties=config)

In [ ]:
seasons_df.write.jdbc(url=jdbc_url, table='seasons', mode=mode, properties=config)

In [ ]:
races_df.write.jdbc(url=jdbc_url, table='races', mode=mode, properties=config)

In [ ]:
pit_stops_df.write.jdbc(url=jdbc_url, table='pit_stops', mode=mode, properties=config)

In [ ]:
lap_times_df.write.jdbc(url=jdbc_url, table='lap_times', mode=mode, properties=config)

In [ ]:
qualifying_df.write.jdbc(url=jdbc_url, table='qualifying', mode=mode, properties=config)

In [ ]:
status_df.write.jdbc(url=jdbc_url, table='status', mode=mode, properties=config)

In [ ]:
results_df.write.jdbc(url=jdbc_url, table='results', mode=mode, properties=config)

In [ ]:
driver_standings_df.write.jdbc(url=jdbc_url, table='driver_standings', mode=mode, properties=config)

In [ ]:
constructor_standings_df.write.jdbc(url=jdbc_url, table='constructor_standings', mode=mode, properties=config)

In [ ]:
constructor_results_df.write.jdbc(url=jdbc_url, table='constructor_results', mode=mode, properties=config)

In [25]:
points_collision_df.write.jdbc(url=jdbc_url, table='points_collision', mode=mode, properties=config)

In [26]:
weather_df.write.jdbc(url=jdbc_url, table='weather', mode=mode, properties=config)